In [150]:
import zipfile
import os

In [151]:
!wget --no-check-certificate \
    "https://github.com/ardamavi/Sign-Language-Digits-Dataset/archive/refs/heads/master.zip" \
    -O "/tmp/Sign-Language-Digits-Dataset-master.zip"


zip_ref = zipfile.ZipFile('/tmp/Sign-Language-Digits-Dataset-master.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2022-12-31 19:45:50--  https://github.com/ardamavi/Sign-Language-Digits-Dataset/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ardamavi/Sign-Language-Digits-Dataset/zip/refs/heads/master [following]
--2022-12-31 19:45:50--  https://codeload.github.com/ardamavi/Sign-Language-Digits-Dataset/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.121.10
Connecting to codeload.github.com (codeload.github.com)|140.82.121.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/Sign-Language-Digits-Dataset-master.zip’

/tmp/Sign-Language-     [      <=>           ]  16.43M  15.5MB/s    in 1.1s    

2022-12-31 19:45:52 (15.5 MB/s) - ‘/tmp/Sign-Language-Digits-Dataset-master.zip’ saved [17231669]



Some Packages

In [152]:
import tensorflow as tf
import sklearn as sk
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Conv2D, MaxPooling2D
from keras.activations import linear, relu, sigmoid
from keras.regularizers import l2
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
%matplotlib inline
import numpy as np
import cv2

Loading the dataset

In [153]:
image_size = 100
num_class = 10
grayscale_images = False

def getImg(path, color):
    img = cv2.imread(path, color)
    img = cv2.resize(img, (image_size, image_size))
    return img

def get_dataset(dataset_path='/tmp/Sign-Language-Digits-Dataset-master/Dataset'):
    labels = os.listdir(dataset_path) # Geting labels
    print(labels)
    X_gray = []
    X_rgb = []
    Y = []
    for i, label in enumerate(labels):
        datas_path = dataset_path+'/'+label
        for data in os.listdir(datas_path): 
            X_gray.append(getImg(datas_path+'/'+data, 0))
            X_rgb.append(getImg(datas_path+'/'+data, 1))
            Y.append(label)
    
    # Normalized gray img:
    X_gray = np.array(X_gray).astype('float32')
    X_gray = X_gray / 255
    

    # Normalized rgb img
    X_rgb = np.array(X_rgb).astype('float32')
    avg = np.average(X_rgb)
    X_rgb = (X_rgb - avg) / 255

    Y = np.array(Y).astype('float32')
    
    return X_gray, X_rgb, Y

X_gray, X_rgb, Y = get_dataset()


print(X_rgb[0])
print(X_rgb[0].shape)
print(X_rgb.shape)


#plt.matshow(X[0])
#plt.imshow(X[0], cmap = "gray")
#plt.show()

['4', '0', '3', '8', '6', '5', '9', '2', '1', '7']
[[[-0.10434828 -0.09258357 -0.07689729]
  [-0.09258357 -0.08081886 -0.06513259]
  [-0.08474043 -0.07297572 -0.05728945]
  ...
  [-0.17493652 -0.17885809 -0.17101495]
  [-0.18277965 -0.18670122 -0.17885809]
  [-0.19062279 -0.19454436 -0.18670122]]

 [[-0.10042671 -0.088662   -0.07297572]
  [-0.088662   -0.07689729 -0.06121102]
  [-0.08081886 -0.06905416 -0.05336788]
  ...
  [-0.16709338 -0.17101495 -0.16317181]
  [-0.17885809 -0.18277965 -0.17493652]
  [-0.18670122 -0.19062279 -0.18277965]]

 [[-0.09258357 -0.08081886 -0.06513259]
  [-0.08474043 -0.07297572 -0.05728945]
  [-0.07297572 -0.06121102 -0.04552475]
  ...
  [-0.15925023 -0.16317181 -0.15532866]
  [-0.17101495 -0.17493652 -0.16709338]
  [-0.17885809 -0.18277965 -0.17493652]]

 ...

 [[-0.17101495 -0.1514071  -0.14748552]
  [-0.16709338 -0.14748552 -0.14356396]
  [-0.16317181 -0.14356396 -0.13964239]
  ...
  [-0.24944632 -0.24160318 -0.23768161]
  [-0.25728944 -0.24944632 -0.245

Model 1: ANN

In [154]:
#splitting the data into train and test and valdiation
X_train, X_test, Y_train, Y_test = train_test_split(X_gray, Y, test_size = 0.2 , random_state = 123)
model1 = Sequential(
    [
        Flatten(input_shape = (100, 100)),
        Dense(1024, activation = "relu"),
        Dense(516, activation = "relu"),
        Dense(256, activation = "relu"),
        Dense(10, activation = "softmax")
    ]
)

model1.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics=['accuracy']
)

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X_train, Y_train):
  model1.fit(X_train[train_index], Y_train[train_index], epochs=15)
  print("\nModel Evaluation")
  model1.evaluate(X_train[test_index], Y_train[test_index])
  print("\n")
  



Epoch 1/15
26/26 [==============================] - 0s 5ms/step - loss: 8.9997 - accuracy: 0.1044
Epoch 2/15
26/26 [==============================] - 0s 6ms/step - loss: 2.4429 - accuracy: 0.1238
Epoch 3/15
26/26 [==============================] - 0s 6ms/step - loss: 2.3656 - accuracy: 0.1408
Epoch 4/15
26/26 [==============================] - 0s 5ms/step - loss: 2.2423 - accuracy: 0.2002
Epoch 5/15
26/26 [==============================] - 0s 5ms/step - loss: 2.1679 - accuracy: 0.2002
Epoch 6/15
26/26 [==============================] - 0s 5ms/step - loss: 2.0217 - accuracy: 0.2998
Epoch 7/15
26/26 [==============================] - 0s 5ms/step - loss: 1.9101 - accuracy: 0.3240
Epoch 8/15
26/26 [==============================] - 0s 5ms/step - loss: 1.7953 - accuracy: 0.3714
Epoch 9/15
26/26 [==============================] - 0s 5ms/step - loss: 1.6128 - accuracy: 0.4296
Epoch 10/15
26/26 [==============================] - 0s 5ms/step - loss: 1.5462 - accuracy: 0.4381
Epoch 11/15
26/26 [

Model 2 ANN Classification Report

In [155]:
y_pred = model1.predict(X_test)

y_pred_labels = [np.argmax(i) for i in y_pred]

print(classification_report(Y_test, y_pred_labels))

13/13 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         0.0       0.66      0.95      0.78        43
         1.0       0.79      0.75      0.77        40
         2.0       0.83      0.63      0.72        38
         3.0       0.77      0.77      0.77        39
         4.0       0.61      0.64      0.62        44
         5.0       0.58      0.90      0.71        39
         6.0       0.77      0.61      0.68        44
         7.0       0.63      0.68      0.65        40
         8.0       0.69      0.66      0.68        38
         9.0       0.88      0.46      0.60        48

    accuracy                           0.70       413
   macro avg       0.72      0.70      0.70       413
weighted avg       0.72      0.70      0.70       413



Model 2: ANN


In [156]:
model2 = Sequential(
    [
        Flatten(input_shape = (100, 100)),
        Dense(1024, activation = "relu"),
        Dense(516, activation = "relu"),
        Dense(10, activation = "softmax")
    ]
)

model2.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics=['accuracy']
)

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X_train, Y_train):
  model2.fit(X_train[train_index], Y_train[train_index], epochs=15)
  print("\nModel Evaluation")
  model2.evaluate(X_train[test_index], Y_train[test_index])
  print("\n")
  

Epoch 1/15
26/26 [==============================] - 0s 5ms/step - loss: 10.4169 - accuracy: 0.1044
Epoch 2/15
26/26 [==============================] - 0s 5ms/step - loss: 2.5184 - accuracy: 0.1335
Epoch 3/15
26/26 [==============================] - 0s 5ms/step - loss: 2.2559 - accuracy: 0.2039
Epoch 4/15
26/26 [==============================] - 0s 5ms/step - loss: 2.1146 - accuracy: 0.2415
Epoch 5/15
26/26 [==============================] - 0s 5ms/step - loss: 1.9758 - accuracy: 0.3070
Epoch 6/15
26/26 [==============================] - 0s 5ms/step - loss: 1.9172 - accuracy: 0.3362
Epoch 7/15
26/26 [==============================] - 0s 5ms/step - loss: 1.7418 - accuracy: 0.4029
Epoch 8/15
26/26 [==============================] - 0s 5ms/step - loss: 1.5739 - accuracy: 0.4223
Epoch 9/15
26/26 [==============================] - 0s 5ms/step - loss: 1.4718 - accuracy: 0.4915
Epoch 10/15
26/26 [==============================] - 0s 5ms/step - loss: 1.2834 - accuracy: 0.5522
Epoch 11/15
26/26 

Model 2 ANN Classification Report

In [157]:
y_pred = model2.predict(X_test)

y_pred_labels = [np.argmax(i) for i in y_pred]

print(classification_report(Y_test, y_pred_labels))

13/13 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         0.0       0.77      0.86      0.81        43
         1.0       0.79      0.82      0.80        40
         2.0       0.70      0.79      0.74        38
         3.0       0.93      0.69      0.79        39
         4.0       0.42      0.66      0.51        44
         5.0       0.84      0.67      0.74        39
         6.0       0.67      0.73      0.70        44
         7.0       0.57      0.75      0.65        40
         8.0       0.80      0.21      0.33        38
         9.0       0.88      0.73      0.80        48

    accuracy                           0.69       413
   macro avg       0.74      0.69      0.69       413
weighted avg       0.73      0.69      0.69       413



CNN Model

In [158]:
#splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X_rgb, Y, test_size = 0.2 , random_state = 123)

cnnModel = Sequential(
    [
        #CNN
        Conv2D(filters=64, kernel_size=(5, 5), activation='relu', input_shape=(100, 100, 3)),
        MaxPooling2D((3, 3)),

        #Dense
        Flatten(),
        Dense(216, activation = "relu"),
        Dense(64, activation = "relu"),
        Dense(10, activation = "softmax")
    ]
)

cnnModel.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics=['accuracy']
)

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X_train, Y_train):
  cnnModel.fit(X_train[train_index], Y_train[train_index], epochs=15)
  print("\nModel Evaluation")
  cnnModel.evaluate(X_train[test_index], Y_train[test_index])
  print("\n")

Epoch 1/15
26/26 [==============================] - 1s 17ms/step - loss: 1.6977 - accuracy: 0.4150
Epoch 2/15
26/26 [==============================] - 0s 17ms/step - loss: 0.6776 - accuracy: 0.7718
Epoch 3/15
26/26 [==============================] - 0s 17ms/step - loss: 0.3307 - accuracy: 0.8981
Epoch 4/15
26/26 [==============================] - 0s 17ms/step - loss: 0.1548 - accuracy: 0.9624
Epoch 5/15
26/26 [==============================] - 0s 17ms/step - loss: 0.0768 - accuracy: 0.9818
Epoch 6/15
26/26 [==============================] - 0s 17ms/step - loss: 0.0425 - accuracy: 0.9927
Epoch 7/15
26/26 [==============================] - 0s 17ms/step - loss: 0.0156 - accuracy: 1.0000
Epoch 8/15
26/26 [==============================] - 0s 17ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 9/15
26/26 [==============================] - 0s 17ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/15
26/26 [==============================] - 0s 17ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 11/

Cnn Model Classification Report

In [159]:
y_pred = cnnModel.predict(X_test)

y_pred_labels = [np.argmax(i) for i in y_pred]


print(classification_report(Y_test, y_pred_labels))

13/13 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.95      0.98        43
         1.0       0.95      0.88      0.91        40
         2.0       0.89      0.82      0.85        38
         3.0       0.90      0.90      0.90        39
         4.0       0.87      0.75      0.80        44
         5.0       0.93      0.97      0.95        39
         6.0       0.71      0.80      0.75        44
         7.0       0.79      0.85      0.82        40
         8.0       0.77      0.79      0.78        38
         9.0       0.84      0.90      0.87        48

    accuracy                           0.86       413
   macro avg       0.86      0.86      0.86       413
weighted avg       0.86      0.86      0.86       413



SVM Model

In [160]:
print(X_train.shape)
print(X_test.shape)

(1649, 100, 100, 3)
(413, 100, 100, 3)


In [162]:
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


svmModel = SVC()
svmModel.fit(X_train, Y_train)

predictions = svmModel.predict(X_test)

SVM Evaluation

In [163]:
print(accuracy_score(Y_test, predictions))
print(classification_report(Y_test, predictions))

0.8305084745762712
              precision    recall  f1-score   support

         0.0       0.89      0.98      0.93        43
         1.0       0.97      0.88      0.92        40
         2.0       0.84      0.84      0.84        38
         3.0       1.00      0.79      0.89        39
         4.0       0.73      0.61      0.67        44
         5.0       0.86      0.95      0.90        39
         6.0       0.74      0.84      0.79        44
         7.0       0.68      0.75      0.71        40
         8.0       0.82      0.82      0.82        38
         9.0       0.84      0.85      0.85        48

    accuracy                           0.83       413
   macro avg       0.84      0.83      0.83       413
weighted avg       0.84      0.83      0.83       413

